In [17]:
import sys
sys.path.insert(1, '../evaluator')

import gc
import copy
import itertools
import random as rndm
import numpy as np
import matplotlib.pyplot as plt
import _pickle as cpickle
from skimage import draw
from inspect import getmembers
from evaluator import *
from pprint import pprint
from tqdm.notebook import tqdm
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

%matplotlib inline

In [18]:
'''
evaluator.py
IMPORTS:

getTurbLoc(TURB_LOC_file_name)

loadPowerCurve(POWER_CURVE_file_name)

binWindResourceData(WIND_DATA_file_name)

searchSorted(lookup, sample_array)

preProcessing(POWER_CURVE)

getAEP(TURB_DIAM, turb_coords, POWER_CURVE, WIND_INST_FREQ,
       N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
       
checkConstraints(turb_coords, TURB_DIAM)

'''

# Turbine Specifications.
# -**-SHOULD NOT BE MODIFIED-**-
TURB_SPECS    =  {
                     'Name': 'Anon Name',
                     'Vendor': 'Anon Vendor',
                     'Type': 'Anon Type',
                     'Dia (m)': 100,
                     'Rotor Area (m2)': 7853,
                     'Hub Height (m)': 100,
                     'Cut-in Wind Speed (m/s)': 3.5,
                     'Cut-out Wind Speed (m/s)': 25,
                     'Rated Wind Speed (m/s)': 15,
                     'Rated Power (MW)': 3
                 }
TURB_DIAM      =  TURB_SPECS['Dia (m)']
TURB_DIAM       =  TURB_DIAM/2

# Load the power curve
POWER_CURVE   =  loadPowerCurve('./../evaluator/power_curve.csv')

# Pass wind data csv file location to function binWindResourceData.
# Retrieve probabilities of wind instance occurence.
WIND_INST_FREQ =  binWindResourceData('./../evaluator/wind_data/wind_data_2007.csv')

# Doing preprocessing to avoid the same repeating calculations. Record
# the required data for calculations. Do that once. Data are set up (shaped)
# to assist vectorization. Used later in function totalAEP.
N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t = preProcessing(POWER_CURVE)

# check if there is any constraint is violated before we do anything. 
def evalPrint(turbine_coordinates):
    checkConstraints(turbine_coordinates, TURB_DIAM)
    
    print('[INFO] Calculating AEP')
    AEP = getAEP(TURB_DIAM, turbine_coordinates, POWER_CURVE, WIND_INST_FREQ,
                 N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
    print('[INFO] Power produced : ', "%.12f"%(AEP), 'GWh')
    
def eval_(turbine_coordinates):
    return getAEP(TURB_DIAM, turbine_coordinates, POWER_CURVE, WIND_INST_FREQ, 
                 N_WIND_INSTANCES,COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
    
    
def create_submission_df(coordinates):
    df = pd.DataFrame(coordinates, columns =['x', 'y'])
    df.to_csv('submission.csv', index=False)
    
# TEST VALUES
# Turbine x,y coordinates
TEST_COORDS   =  getTurbLoc(r'./../test_locations.csv')

In [19]:
%%time
# SELF TEST
_ = checkConstraints(TEST_COORDS, TURB_DIAM)
print("--------######---------")
evalPrint(TEST_COORDS)
print()

[SUCCESS] perimeter and proximity constraints - SATISFIED
--------######---------
[SUCCESS] perimeter and proximity constraints - SATISFIED
[INFO] Calculating AEP
[INFO] Power produced :  505.450636596680 GWh

CPU times: user 130 ms, sys: 15.7 ms, total: 146 ms
Wall time: 146 ms


## Classes

In [20]:
class DNA:
    """
    init
    gen_init_state
    
    LOOP:
        calc_fitness
        mutate
        sort_tuples

    FINALLY
    update_current_state
    """
    ###########################################
    def __init__(self):
        self.fitness = 0
        self.genes, self.state = self.generate_init_state()
       

    ###########################################
    # GENERATE SET OF VALID GENES
    def generate_init_state(self):
        state = np.zeros((3900, 3900)).astype(np.int8)
        coordinates = np.array([], dtype=np.int32).reshape(0,2)

        count_placed = 0
        while(count_placed < 50):
            free_coordinates = np.where(state == 0)
            if(len(free_coordinates[0]) == 0): 
                return -1

            free_coordinates = np.stack([*free_coordinates]).transpose()
            chosen_coordinate = rndm.choice(free_coordinates)
            
            r, c = draw.circle(chosen_coordinate[0], chosen_coordinate[1], radius=400, shape=state.shape)
            state[r, c] = 1
            r, c = draw.circle(chosen_coordinate[0], chosen_coordinate[1], radius=50, shape=state.shape)
            state[r, c] = 2

            coordinates = np.concatenate([coordinates, chosen_coordinate.reshape(1,2)], axis=0)
            count_placed += 1

        coordinates = coordinates + 50
        coordinates = coordinates[np.lexsort((coordinates[:,1],coordinates[:,0]))]
        
        return (coordinates.flatten(),  np.pad(state, 50))

    # UPDATE CURRENT STATE
    def update_current_state(self):
        coordinates = self.get_tuples()
        state = np.zeros((4000, 4000)).astype(np.int8)
        for coord in coordinates:
            r, c = draw.circle(coord[0], coord[1], radius=400, shape=state.shape)
            state[r, c] = 1
            r, c = draw.circle(coord[0], coord[1], radius=50, shape=state.shape)
            state[r, c] = 2
        self.state = state
    
    
    ###########################################
    # CALC & STORE FITNESS
    def calc_fitness(self):
        if(checkConstraints_F(self.get_tuples(), TURB_DIAM) == -1):
            return 450
        self.fitness = getAEP(TURB_DIAM, self.get_tuples(), POWER_CURVE, WIND_INST_FREQ, 
                              N_WIND_INSTANCES,COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
        return self.fitness
    
    ###########################################
    # MUTATE 
    def mutate(self, mutation_rate):
        for i in range(100):  # 100 genes (if grouped as 50 pairs, use 50)
            if(rndm.uniform(0,1) <= mutation_rate):
                mutant = rndm.randint(0,60)
                if(rndm.uniform(0,1) > 0.5):
                    self.genes[i] += mutant
                else:
                    self.genes[i] -= mutant
        return True
            

    ###########################################
    # GET DNA AS TUPLE OF COORDINATES (IF FLATTENED)
    def get_tuples(self):
        return np.reshape(self.genes, newshape=(50,2))
    
    def sort_tuples(self):
        coordinates = self.get_tuples()
        coordinates = coordinates[np.lexsort((coordinates[:,1],coordinates[:,0]))]
        self.genes = coordinates.flatten()
    
    def __getitem__(self, index):
        print("[ERR] USE d.genes[i]")

In [21]:
class Population:
    """
    init
    
    LOOP:
        calc_fitnesses
        create_crossover
        create_mutation
            (DNA) mutate
        create_next_generation
            (DNA) sort_tuples
            
    FINALLY (selected_DNA)
    (DNA) update_current_state
    """
    ###########################################
    def __init__(self, population_size, mutation_rate):
        self.best_DNA = None
        self.max_fitness = 0
        self.gen_max_fitness = 0
        self.gen_min_fitness = 0
        self.population_fitnesses = None
        self.generations = 0
        
        self.population_size = population_size
        self.mutation_rate = mutation_rate

        # Multithreaded population init
        processes = []
        with ProcessPoolExecutor(max_workers=4) as executor:
            for _ in range(self.population_size): 
                processes.append(executor.submit(DNA))
        self.population = np.array([processes[i].result() for i in range(self.population_size)])
        self.next_population = None
    
    
    ###########################################
    # CALC & GET FITNESS   
    def calc_fitnesses(self):
        fitnesses = np.array([self.population[i].calc_fitness() for i in range(self.population_size)])
        self.population_fitnesses = fitnesses
        self.gen_min_fitness = np.min(fitnesses)
        max_index = np.argmax(fitnesses)
        self.gen_max_fitness = fitnesses[max_index]
        if(fitnesses[max_index] > self.max_fitness):
            self.best_DNA = self.population[max_index]
            self.max_fitness = fitnesses[max_index]
            
            print(f'NEW MAX:  {self.max_fitness}')
            with open("BEST_DNA.pickle", "wb") as output_file:
                cpickle.dump(self.best_DNA, output_file)
        return fitnesses
        
    def get_fitnesses(self):
        return np.array([self.population[i].fitness for i in range(self.population_size)])
    
    
    #############################################
    # NATURAL SELECTION   
    def create_crossover(self):
        weights = (self.population_fitnesses - self.gen_min_fitness) / (self.max_fitness - self.gen_min_fitness)
        if(np.sum(weights) == 0):
            weights = weights + 1
        next_population = copy.deepcopy(self.population)
        for i in range(self.population_size):
            cur_genes = []
            parents = rndm.choices(self.population, weights=weights, k=2)
            for j in range(100):  # 100 genes (if grouped as 50 pairs, use 50)
                if(rndm.uniform(0,1) > 0.5):
                    cur_genes.append(parents[0].genes[j])
                else:
                    cur_genes.append(parents[1].genes[j])
            next_population[i].genes = np.array(cur_genes)
        self.next_population = next_population
        
    # MUTATE GENES
    def create_mutation(self):
        processes = []
        with ProcessPoolExecutor(max_workers=4) as executor:
            for i in range(self.population_size):
                processes.append(executor.submit(self.next_population[i].mutate, self.mutation_rate))
        status = all([processes[i].result() for i in range(self.population_size)])
        return status
    
    # UPDATE GENERATION
    def create_next_generation(self):
        for i in range(self.population_size):
            self.next_population[i].sort_tuples()
        self.population = self.next_population
        self.generations += 1
        

    #############################################
    # GET DNAs            
    def get_population_coordinates(self):
        return np.array([self.population[i].get_tuples() for i in range(self.population_size)])
        
    def __getitem__(self, index):
        return self.population[index]

## Main

In [22]:
with open("BEST_DNA.pickle", "rb") as input_file:
    BEST_DNA = cpickle.load(input_file)

In [25]:
pprint(getmembers(BEST_DNA))

[('__class__', <class '__main__.DNA'>),
 ('__delattr__', <method-wrapper '__delattr__' of DNA object at 0x821f627d0>),
 ('__dict__',
  {'fitness': 513.2880670166015,
   'genes': array([ 151,  772,  203, 3766,  213, 1476,  246, 2640,  275,  148,  299,
       3062,  632, 1059,  671, 3532,  711,   61,  886,  600,  943, 1746,
        963, 3099, 1060, 2401, 1150,   72, 1284, 1231, 1349, 2935, 1481,
       3907, 1487, 1624, 1489, 3499, 1500,  670, 1520, 2307, 1603,  266,
       1941, 1332, 1942, 2230, 2052, 3511, 2144,  554, 2195, 3899, 2246,
       2739, 2266,  126, 2324, 1679, 2374, 1252, 2584,  626, 2661, 3270,
       2666, 2362, 2701, 1520, 2867,  958, 2869,  118, 2891, 3637, 2978,
       2771, 3170, 1849, 3254,  564, 3297, 2479, 3310, 1136, 3349, 3861,
       3498, 3341, 3540, 1617, 3694, 2559, 3751, 3667, 3847,  864, 3948,
         83]),
   'state': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0]